In [1]:
import numpy as np
import pandas as pd
from wedpr_ml_toolkit.config.wedpr_ml_config import WeDPRMlConfigBuilder
from wedpr_ml_toolkit.wedpr_ml_toolkit import WeDPRMlToolkit
from wedpr_ml_toolkit.toolkit.dataset_toolkit import DatasetToolkit

['/usr/lib/python3/dist-packages/wedpr_ml_toolkit/', 'd:\\github\\wedpr3.0\\WeDPR-Component\\python\\wedpr_ml_toolkit', 'd:\\github\\wedpr3.0\\WeDPR-Component\\python', 'd:\\github\\wedpr3.0\\WeDPR-Component\\python', 'c:\\Users\\yanxi\\anaconda3\\python38.zip', 'c:\\Users\\yanxi\\anaconda3\\DLLs', 'c:\\Users\\yanxi\\anaconda3\\lib', 'c:\\Users\\yanxi\\anaconda3', '', 'c:\\Users\\yanxi\\anaconda3\\lib\\site-packages', 'c:\\Users\\yanxi\\anaconda3\\lib\\site-packages\\win32', 'c:\\Users\\yanxi\\anaconda3\\lib\\site-packages\\win32\\lib', 'c:\\Users\\yanxi\\anaconda3\\lib\\site-packages\\Pythonwin', 'c:\\Users\\yanxi\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\yanxi\\.ipython']


In [2]:
# 读取配置文件
wedpr_config = WeDPRMlConfigBuilder.build_from_properties_file('config.properties')
wedpr_ml_toolkit = WeDPRMlToolkit(wedpr_config)

In [3]:
# 注册 dataset，支持两种方式: pd.Dataframe, hdfs_path
# 1. pd.Dataframe
df = pd.DataFrame({
    'id': np.arange(0, 100),  # id列，顺序整数
    'y': np.random.randint(0, 2, size=100),
    # x1到x10列，随机数
    **{f'x{i}': np.random.rand(100) for i in range(1, 11)}
})

dataset1 = DatasetToolkit(storage_entrypoint=wedpr_ml_toolkit.get_storage_entry_point(),
                          storage_workspace=wedpr_config.user_config.get_workspace_path(),
                          dataset_owner='flyhuang1',
                          agency=wedpr_config.user_config.agency_name,
                          values=df,
                          is_label_holder=True)
print(dataset1.storage_client.storage_client.endpoint, dataset1.storage_workspace, dataset1.agency)
dataset1.storage_client = None  # 本地测试时跳过hdfs上传/下载过程
dataset1.save_values(path='d-101')
print(dataset1.dataset_path)
print(dataset1.values.head())

http://139.159.202.235:50070 /user/ppc/milestone2/sgd/test_user SGD
/user/ppc/milestone2/sgd/test_user\d-101
   id  y        x1        x2        x3        x4        x5        x6  \
0   0  1  0.954183  0.652034  0.704070  0.180889  0.025025  0.511596   
1   1  1  0.302088  0.462222  0.435542  0.029966  0.931294  0.848483   
2   2  1  0.468104  0.430161  0.239322  0.588153  0.470668  0.225856   
3   3  0  0.152269  0.811666  0.834451  0.354288  0.635447  0.062092   
4   4  0  0.841470  0.800512  0.451507  0.118651  0.748845  0.557916   

         x7        x8        x9       x10  
0  0.529848  0.759689  0.159081  0.556419  
1  0.962787  0.224096  0.464418  0.208487  
2  0.564879  0.730366  0.394245  0.299081  
3  0.424057  0.202234  0.577448  0.636958  
4  0.030906  0.514350  0.340864  0.123303  


In [4]:
# 2. hdfs_path
dataset2 = DatasetToolkit(storage_entrypoint=wedpr_ml_toolkit.get_storage_entry_point(), 
                          storage_workspace=wedpr_config.user_config.get_workspace_path(), 
                          dataset_owner='flyhuang',
                          dataset_path="/user/ppc/milestone2/webank/flyhuang/d-9606695119693829", 
                          agency="WeBank")
print(dataset2.storage_client.storage_client.endpoint, dataset2.storage_workspace, dataset2.agency)
print(dataset2.dataset_path)
dataset2.storage_client = None  # 本地测试时跳过hdfs上传/下载过程

# 提供本地测试数据
if dataset2.storage_client is None:
    # 支持更新dataset的values数据
    df2 = pd.DataFrame({
        'id': np.arange(0, 100),  # id列，顺序整数
        **{f'z{i}': np.random.rand(100) for i in range(1, 11)}  # x1到x10列，随机数
    })
    dataset2.update_values(values=df2)
    dataset2.save_values()
    print(dataset2.dataset_path)
    print(dataset2.values.head())

# 对于己方数据集支持load_values，其他方数据集无需load_values，可直接使用
if dataset2.storage_client is not None:
    # 仅支持load本机构hdfs的数据集
    dataset2.load_values(header=0)
    print(dataset2.dataset_path)
    print(dataset2.values.head())

http://139.159.202.235:50070 /user/ppc/milestone2/sgd/test_user WeBank
/user/ppc/milestone2/webank/flyhuang/d-9606695119693829
/user/ppc/milestone2/webank/flyhuang/d-9606695119693829
   id        z1        z2        z3        z4        z5        z6        z7  \
0   0  0.597205  0.942475  0.886443  0.560584  0.254432  0.370152  0.076031   
1   1  0.778616  0.607374  0.616211  0.602282  0.385989  0.816963  0.756814   
2   2  0.999795  0.596794  0.240741  0.241070  0.857676  0.342412  0.066459   
3   3  0.968410  0.895163  0.636140  0.978791  0.237098  0.095272  0.938806   
4   4  0.921513  0.454901  0.004514  0.769216  0.627185  0.676253  0.184952   

         z8        z9       z10  
0  0.587627  0.851390  0.864929  
1  0.661537  0.865674  0.050091  
2  0.473916  0.080120  0.477873  
3  0.452399  0.953515  0.405465  
4  0.877475  0.316322  0.139290  


In [5]:
# 更新数据集
if dataset1.storage_client is not None:
    dataset1.update_values(
        path='/user/ppc/milestone2/sgd/flyhuang1/d-9606704699156485')
    dataset1.load_values(header=0)
print(dataset1.dataset_path)
print(dataset1.values.head())

/user/ppc/milestone2/sgd/test_user\d-101
   id  y        x1        x2        x3        x4        x5        x6  \
0   0  1  0.954183  0.652034  0.704070  0.180889  0.025025  0.511596   
1   1  1  0.302088  0.462222  0.435542  0.029966  0.931294  0.848483   
2   2  1  0.468104  0.430161  0.239322  0.588153  0.470668  0.225856   
3   3  0  0.152269  0.811666  0.834451  0.354288  0.635447  0.062092   
4   4  0  0.841470  0.800512  0.451507  0.118651  0.748845  0.557916   

         x7        x8        x9       x10  
0  0.529848  0.759689  0.159081  0.556419  
1  0.962787  0.224096  0.464418  0.208487  
2  0.564879  0.730366  0.394245  0.299081  
3  0.424057  0.202234  0.577448  0.636958  
4  0.030906  0.514350  0.340864  0.123303  
